In [1]:
from nixtlats import TimeGPT
import pandas as pd
import numpy as np
import torch 
import config
import pickle

timegpt = TimeGPT(
    # defaults to os.environ.get("TIMEGPT_TOKEN")
    token = 'JxQF7I2vM8Hpbue718L2gGCfG1gflGqWJCkWb6gRjVL2r9p0TPhDmoOIMAAOgGdQG1tFUDFKgkq52cbyAM1H8Li1jTg9KtV6zWKGuejojqMV6i5wzTgppL8KnsvfcfpYidXbBRgqSLlVilfjjvWwTIMFixvHbIu1cEBdP8CUcjeRlsWlg2ZkYlLr3iiG2RojTAG4BKHRnSX2S7uIpaKMjBZcwRlVpf8KoUFXfAaQj2a5UW98o5IMPe196Iz2uqwD'
)


Nixtla's SDK for TimeGPT takes pandas DF as inputs. Here we are using an Electricity Price Forecasting (EPF) dataset.

The input data will require the following columns:

series identifier (column name specified with #id_col argument)
datestamps (column name specified with #time_col argument)
target historical values (column name specified with #target_col argument)

FINE TUNING IS ONLY POSSIBLE STARTING AT 1.5X FORECASTING HORIZON


MSE: 

(2.4750)

(0.0772)

(0.0770)

after forecasting 6 horizons at 96 time steps for 350 IDs, 1000 dollars of forecasting credit are used and no more forecsats can be made.  

In [2]:
timegpt.validate_token()


ApiError: status_code: 429, body: {'status': 429, 'data': None, 'message': 'Too many requests', 'details': 'You need to add a payment method to continue using the API, do so from https://dashboard.nixtla.io', 'code': 'A21', 'support': 'If you have questions or need support, please email ops@nixtla.io', 'requestID': '3XH898UV4N'}

In [4]:
from utils import data_handling, helpers

window_size = 96
pred_length = pred_length = (96)#, 192, 336, 720)

data_dict = data_handling.load_electricity()

In [6]:
def input_tensor_to_gpt_df(input):
    horizon = input.size(0)
    num_ids = input.size(1)


    timestamps = pd.date_range(start='2023-04-01', periods=horizon, freq='H')
    time_steps = np.repeat(timestamps, num_ids)

    ids = np.tile(np.arange(0, num_ids), horizon) 


    input = input.numpy().flatten()


    df = pd.DataFrame({
    'Time': time_steps,
    'ID': ids,
    'Target': input
    })
    
    return df


def reshape_gpt_output_to_eval_tensor(timegpt_fcst_df):
    temp = timegpt_fcst_df

    num_ids = temp["ID"].nunique()
    horizon = temp["Time"].nunique()

    preds = temp["TimeGPT"]
    preds_tensor = torch.tensor(preds.to_list())
    preds_tensor_2d = preds_tensor.view(horizon, num_ids)
    

    return preds_tensor_2d

In [11]:
prediction_list = []
num_96_horizons = int(data_dict["test"].shape[0] / 96)


filename = config.CONFIG_OUTPUT_PATH["TimeGPT"] / "TimeGPT_zero_shot_electricity.pkl"

for i in range(1, num_96_horizons):
    time_step = i * 96
    sum_mse = 0
    print(i)
    input = data_dict["test"][time_step:time_step+96,:]
    target = data_dict["test"][time_step+96: time_step+96+96, :]

    input_df = input_tensor_to_gpt_df(input)
    timegpt_fcst_df = timegpt.forecast(df=input_df, h=96, freq="H", model="timegpt-1-long-horizon", finetune_steps=0, id_col='ID', time_col='Time', target_col='Target')

    preds = reshape_gpt_output_to_eval_tensor(timegpt_fcst_df)

    sum_mse += helpers.mean_squared_error(preds, target)


    prediction_list.append(sum_mse / num_96_horizons)
    print(sum_mse / num_96_horizons)

        # save as pickle
    with open(filename, 'wb') as file:
        pickle.dump(prediction_list, file)

INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...


1


INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...


tensor(0.0772)
2


INFO:nixtlats.timegpt:Calling Forecast Endpoint...
INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...


tensor(0.0770)
3


ApiError: status_code: 429, body: {'status': 429, 'data': None, 'message': 'Too many requests', 'details': 'You need to add a payment method to continue using the API, do so from https://dashboard.nixtla.io', 'code': 'A21', 'support': 'If you have questions or need support, please email ops@nixtla.io', 'requestID': 'T3C5TMA2LH', 'deprecation_details': 'The current endpoint /timegpt_model_params will be deprecated on 2024-06-21, the new path will be /model_params please refer to the docs (https://docs.nixtla.io/reference) for more info or send an email to ops@nixtla.io'}

In [33]:
prediction_list = []
num_96_horizons = int(data_dict["test"].shape[0] / 96)
num_96_horizons = 2

for i in range(num_96_horizons):
    time_step = i * 96
    sum_mse = 0
    print(i)
    input = data_dict["test"][time_step:time_step+96,:]
    target = data_dict["test"][time_step+96: time_step+96+96, :]

    input_df = input_tensor_to_gpt_df(input)
    timegpt_fcst_df = timegpt.forecast(df=input_df, h=96, freq="H", finetune_steps=4, id_col='ID', time_col='Time', target_col='Target')

    preds = reshape_gpt_output_to_eval_tensor(timegpt_fcst_df)

    sum_mse += helpers.mean_squared_error(preds, target)


    prediction_list.append(sum_mse / num_96_horizons)
    print(sum_mse / num_96_horizons)

INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...


0


Exception: Your time series data is too short Please be sure that your unique time series contain at least 144 observations